In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import jaccard_score

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/Deep Learning projekt")

Mounted at /content/drive


In [3]:
test_sentences = np.load("data/test_sentences.npy").tolist()
test_idxs = np.load("data/test_idxs.npy")

In [4]:
print(test_sentences[0])
print(test_sentences[test_idxs[0]])

What is The main difference between PRV & PSV in pipeline?
What is the main difference between PRV and PSV in pipeline?


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words("english"))
stop_words.add("?")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
test_sentences_lc = [s.lower() for s in test_sentences]
corpus = []
for s in test_sentences_lc:
  corpus.append([word for word in word_tokenize(s) if word not in stop_words])
allwords = list(set([word for sentence in corpus for word in sentence]))

In [7]:
n_words = len(allwords)
size_corpus = len(corpus)
bag_of_words = np.zeros((size_corpus, n_words))
for i, sentence in enumerate(corpus):
  for word in sentence:
    if word in allwords:
      bag_of_words[i, allwords.index(word)] = 1

In [12]:
def jaccard(x, y):
  score = np.dot(x,y) / (np.dot(x,x) + np.dot(y,y) - np.dot(x,y))
  return score

j_scores = np.zeros((size_corpus, size_corpus))
for i in range(size_corpus):
  for j in range(size_corpus):
    if i == j:
      j_scores[i,j] = 0
      continue
    j_scores[i, j] = jaccard(bag_of_words[i], bag_of_words[j])

  if i % 10 == 0:
    print('\r', end='')
    print(i, '/', size_corpus, end='')
np.save('jaccard_scores.npy', j_scores)

10480 / 10490

In [13]:
trues = np.zeros(5)
for i in range(size_corpus):
  true_dup = test_idxs[i]
  top_idxs = j_scores[i].argsort()[-5:][::-1]
  if true_dup in top_idxs:
    pos = top_idxs.tolist().index(true_dup)
    for k in range(5):
      if pos <= k:
        trues[k] += 1

accs = trues / size_corpus
print(accs)

[0.85367016 0.90285987 0.93288847 0.94680648 0.95614871]
